In [94]:
#classification implementation code --- to be cleaned up and pasted into Technical Report

import importlib
from tabulate import tabulate

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

random_state_val = 0

In [60]:
#loading data
stroke_data = MyPyTable()
stroke_data.load_from_file("input_data/stroke-data-downsized.csv")
print("attribute names:", stroke_data.column_names)
print("len before removing missing vals", len(stroke_data.data))

#code to remove rows w/missing values (use only once relevant attributes are decided)
#stroke_data.remove_rows_with_missing_values()
#stroke_data.remove_rows_with_missing_values_by_col()
#print("len after removing missing vals", len(stroke_data.data))

#code to group data by class
stroke_classes, stroke_data_by_class = myutils.group_by(stroke_data.data, stroke_data.column_names, "stroke")
print("classes:", stroke_classes)
for partition in stroke_data_by_class:
    print("num instances of class", partition[0][-1], ":", len(partition))



attribute names: ['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']
len before removing missing vals 1000
classes: [0.0, 1.0]
num instances of class 0.0 : 791
num instances of class 1.0 : 209


In [74]:
#make classifiers - KNN, NB, DT, RF
knn_clf = MyKNeighborsClassifier(n_neighbors=5)
nb_clf = MyNaiveBayesClassifier()
tree_clf = MyDecisionTreeClassifier()
#TODO: random forest

#clean stroke data for classification- discretize, convert nominal to numeric
stroke_data_discretized = myutils.discretize_attributes_for_stroke_classification(stroke_data)
stroke_data_discretized.save_to_file("input_data/stroke-data-discretized.csv")

#strings to numeric
stroke_data_cleaned_numeric = myutils.numerize_all_strings(stroke_data_discretized)
stroke_data_cleaned_numeric.save_to_file("input_data/stroke-data-cleaned-numeric.csv")

#Create X and y data
X = [inst[:-1] for inst in stroke_data_cleaned_numeric.data]
y = [inst[-1] for inst in stroke_data_cleaned_numeric.data]

In [95]:
#k fold cross validation
knn_y_pred = []
nb_y_pred = []
tree_y_pred = []
#forest_y_pred = []

y_true = []
X_train_folds, X_test_folds = myevaluation.kfold_cross_validation(X, n_splits=10, random_state=random_state_val, shuffle=True)
for fold_index in range(len(X_train_folds)): 
    X_train, y_train, X_test, y_test = myutils.one_fold_splits(X, y, X_train_folds, X_test_folds, fold_index)
    #build y true
    for val in y_test:
        y_true.append(val)
   
    #fit and predict kNN classifier
    knn_clf.fit(X_train, y_train)
    for pred in knn_clf.predict(X_test):
        knn_y_pred.append(pred) 
   
   #fit and predict naive bayes
    nb_clf.fit(X_train, y_train)
    for pred in nb_clf.predict(X_test):
        nb_y_pred.append(pred)
   
   #fit and predict tree    #TODO ASK ABOUT THIS ERROR MESSAGE
    #tree_clf.fit(X_train, y_train)
    #for pred in tree_clf.predict(X_test):
    #    tree_y_pred.append(pred)

In [96]:
predictions = [knn_y_pred, nb_y_pred] #TODO add tree, forest results
titles = ["kNN Classifier (5 neighbors)", "Naive Bayes"]

for i in range(len(predictions)):
    myutils.show_results(y_true, predictions[i], stroke_classes, titles[i])


kNN Classifier (5 neighbors)
Accuracy:	 0.74
Error Rate:	 0.26
Precision:	 0.78
Recall:		 0.93
F1 measure:	 0.85
Confusion matrix:
       0.0    1.0
--  -----  -----
 0    739     52
 1    204      5

Naive Bayes
Accuracy:	 0.79
Error Rate:	 0.21
Precision:	 0.79
Recall:		 1.0
F1 measure:	 0.88
Confusion matrix:
       0.0    1.0
--  -----  -----
 0    791      0
 1    209      0
